# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [2]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [3]:
# load the dataset
path= "fgraph_eigen_9.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,...,FGRAPH_EIGEN4,FGRAPH_EIGEN5,FGRAPH_EIGEN6,FGRAPH_EIGEN7,FGRAPH_EIGEN8,FGRAPH_EIGEN9,FGRAPH_EIGEN10,FGRAPH_EIGEN11,FGRAPH_EIGEN12,FGRAPH_EIGEN13
0,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000e+00,-1.000000e+00,-4.142136e-01,-1.915442e-01,...,-1.854805,-1.338680,-0.792572,-7.824513e-01,2.430154e-01,0.511091,2.849471,3.087488,3.556025,4.000000
1,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000e+00,-1.000000e+00,-4.142136e-01,-1.915442e-01,...,-2.419305,-1.070507,-0.795282,-1.541898e-01,1.418155e-01,0.490812,2.786157,3.107490,3.419152,4.000000
2,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000e+00,-1.000000e+00,-4.142136e-01,-1.915442e-01,...,-2.144987,-1.456231,-0.973549,-3.875769e-02,3.459959e-01,0.413189,2.651817,3.137501,3.534380,4.000000
3,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000e+00,-1.000000e+00,-4.142136e-01,-1.915442e-01,...,-2.197858,-1.634606,-0.549630,-1.894592e-01,-7.032175e-03,0.416147,2.910382,3.128880,3.456286,4.000000
4,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000e+00,-1.000000e+00,-4.142136e-01,-1.915442e-01,...,-2.052033,-1.389435,-1.194034,-2.899062e-01,3.410076e-01,0.361708,2.657169,2.971803,3.555856,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43012,0,-2.800647,-2.355674,-2.216389,-1.477260,-1.336072,-8.384353e-01,-2.623597e-01,1.526139e-16,3.560807e-01,...,-1.262864,-1.195535,-0.653701,-4.918611e-01,1.963255e-01,0.573179,1.832476,2.804245,4.000000,4.129357
43013,0,-2.618034,-2.230725,-2.181780,-1.618034,-1.059752,-4.828592e-01,-3.819660e-01,-1.499456e-18,4.455887e-16,...,-1.302776,-1.254102,-0.697224,-8.954799e-16,7.301450e-18,0.486456,1.114908,2.302776,4.000000,4.387619
43014,0,-2.461929,-2.000000,-2.000000,-2.000000,-1.168300,-2.286111e-15,-9.848175e-16,-4.114768e-16,-1.169372e-16,...,-1.618034,-1.254102,-0.381966,-8.937151e-16,5.536663e-18,0.486456,0.618034,1.114908,4.000000,4.387619
43015,0,-2.773959,-2.532089,-1.828224,-1.691229,-1.347296,-9.571429e-01,-2.956744e-01,-5.563084e-16,6.496113e-01,...,-1.430204,-1.090978,-0.502364,-9.066894e-16,9.066894e-16,0.749582,2.361178,2.654548,4.000000,4.000000


In [4]:
# Let's round up the columns to 6 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 6 decimals
df[cols_to_round] = df[cols_to_round].round(6)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,...,FGRAPH_EIGEN4,FGRAPH_EIGEN5,FGRAPH_EIGEN6,FGRAPH_EIGEN7,FGRAPH_EIGEN8,FGRAPH_EIGEN9,FGRAPH_EIGEN10,FGRAPH_EIGEN11,FGRAPH_EIGEN12,FGRAPH_EIGEN13
0,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000,-1.000000,-0.414214,-0.191544,...,-1.854805,-1.338680,-0.792572,-0.782451,0.243015,0.511091,2.849471,3.087488,3.556025,4.000000
1,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000,-1.000000,-0.414214,-0.191544,...,-2.419305,-1.070507,-0.795282,-0.154190,0.141816,0.490812,2.786157,3.107490,3.419152,4.000000
2,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000,-1.000000,-0.414214,-0.191544,...,-2.144987,-1.456231,-0.973549,-0.038758,0.345996,0.413189,2.651817,3.137501,3.534380,4.000000
3,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000,-1.000000,-0.414214,-0.191544,...,-2.197858,-1.634606,-0.549630,-0.189459,-0.007032,0.416147,2.910382,3.128880,3.456286,4.000000
4,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000,-1.000000,-0.414214,-0.191544,...,-2.052033,-1.389435,-1.194034,-0.289906,0.341008,0.361708,2.657169,2.971803,3.555856,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43012,0,-2.800647,-2.355674,-2.216389,-1.477260,-1.336072,-0.838435,-0.262360,0.000000,0.356081,...,-1.262864,-1.195535,-0.653701,-0.491861,0.196326,0.573179,1.832476,2.804245,4.000000,4.129357
43013,0,-2.618034,-2.230725,-2.181780,-1.618034,-1.059752,-0.482859,-0.381966,-0.000000,0.000000,...,-1.302776,-1.254102,-0.697224,-0.000000,0.000000,0.486456,1.114908,2.302776,4.000000,4.387619
43014,0,-2.461929,-2.000000,-2.000000,-2.000000,-1.168300,-0.000000,-0.000000,-0.000000,-0.000000,...,-1.618034,-1.254102,-0.381966,-0.000000,0.000000,0.486456,0.618034,1.114908,4.000000,4.387619
43015,0,-2.773959,-2.532089,-1.828224,-1.691229,-1.347296,-0.957143,-0.295674,-0.000000,0.649611,...,-1.430204,-1.090978,-0.502364,-0.000000,0.000000,0.749582,2.361178,2.654548,4.000000,4.000000


# ChatGpt clasifier computation

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [6]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [7]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8389121338912134
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.92      0.90      6499
           1       0.71      0.58      0.64      2105

    accuracy                           0.84      8604
   macro avg       0.79      0.75      0.77      8604
weighted avg       0.83      0.84      0.83      8604



### most important feature

In [21]:
# 7. Get feature importance from Logistic Regression
feature_names = X.columns
coefficients = clf.coef_[0]  # For binary classification, it's a 1D array

# Create a DataFrame for sorting and readability
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
})

# Sort by absolute value of coefficients
feature_importance_sorted = feature_importance.sort_values(by='abs_coefficient', ascending=False)

# Display the most relevant feature
most_relevant = feature_importance_sorted.iloc[0:5]
print("Most relevant features:")
print(most_relevant)


Most relevant features:
           feature  coefficient  abs_coefficient
10     DEN_EIGEN11    -2.278267         2.278267
11     DEN_EIGEN12     1.524199         1.524199
22  FGRAPH_EIGEN11    -1.484623         1.484623
0       DEN_EIGEN1     1.160693         1.160693
21  FGRAPH_EIGEN10    -1.095030         1.095030


### Decision tree

In [22]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7472324723247232
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.80      0.79       330
           1       0.68      0.67      0.67       212

    accuracy                           0.75       542
   macro avg       0.73      0.73      0.73       542
weighted avg       0.75      0.75      0.75       542



In [23]:
# 4. Feature importances from the decision tree
importances = tree_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
           feature  importance
22  FGRAPH_EIGEN11    0.186502
10     DEN_EIGEN11    0.168651
21  FGRAPH_EIGEN10    0.141105
11     DEN_EIGEN12    0.057904


### Random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.8694793119479312
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.94      0.92      6499
           1       0.78      0.65      0.71      2105

    accuracy                           0.87      8604
   macro avg       0.84      0.80      0.81      8604
weighted avg       0.86      0.87      0.87      8604



In [26]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
           feature  importance
22  FGRAPH_EIGEN11    0.117148
10     DEN_EIGEN11    0.114370
21  FGRAPH_EIGEN10    0.107355
18   FGRAPH_EIGEN7    0.058450


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [9]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.8325545714484971


In [32]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


SVM accuracy on full dataset: 0.9121447028423773


Polynomial fit

In [33]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.8600959763750461


## Random forest  perfect score

We see that the random forest reach a prefect score. If we remove the denominator eigenvalues it reaces an almost perfect score

In [10]:
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 1.0


In [46]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
            feature  importance
10  FGRAPH_EIGEN11    0.161398
9   FGRAPH_EIGEN10    0.139988
8    FGRAPH_EIGEN9    0.124719
0    FGRAPH_EIGEN1    0.089089
6    FGRAPH_EIGEN7    0.087516


In [47]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN",'DEN_EIGEN1', 'DEN_EIGEN2', 'DEN_EIGEN3',
       'DEN_EIGEN4', 'DEN_EIGEN5', 'DEN_EIGEN6', 'DEN_EIGEN7', 'DEN_EIGEN8',
       'DEN_EIGEN9', 'DEN_EIGEN10', 'DEN_EIGEN11', 'DEN_EIGEN12'])
y = df["COEFFICIENT_DEN"]


# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [48]:
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 0.9977851605758582


In [63]:
# 5. Find misclassified samples
misclassified = df[y != y_pred_rf]

print(f"Number of misclassified samples: {len(misclassified)}")
print("Misclassified indices:")
print(misclassified.index.tolist())

Number of misclassified samples: 6
Misclassified indices:
[151, 244, 921, 1467, 1484, 1486]


In [42]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())

Top features:
            feature  importance
10  FGRAPH_EIGEN11    0.161398
9   FGRAPH_EIGEN10    0.139988
8    FGRAPH_EIGEN9    0.124719
0    FGRAPH_EIGEN1    0.089089
6    FGRAPH_EIGEN7    0.087516
